Este cuaderno es una demo del modulo _Simulador_ y el modulo de optizacion (_Workforce_), que usa este para computar una estrategia optima. 

# Simulador (noviembre 2023)
## para simulación: simv05


In [1]:
import os
os.chdir('/DeepenData/Repos/Flux_v0')

from   src.utils_Escritoriosv05_Simv05 import (
                                           generar_planificacion,
                                            DatasetTTP)

from dev.Escritoriosv05_Simv05 import simv05


dataset                                 = DatasetTTP.desde_csv_atenciones("data/fonasa_monjitas.csv.gz")
un_dia                                  = dataset.un_dia("2023-05-15").sort_values(by='FH_Emi', inplace=False)
planificacion, niveles_servicio_x_serie = generar_planificacion(un_dia)
hora_cierre="16:00:00"
registros_atenciones, fila =  simv05(un_dia, hora_cierre, planificacion, niveles_servicio_x_serie)

/DeepenData/Repos/Flux_v0/src/utils_Escritoriosv05_Simv05.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Priority'] = df['IdSerie'].map(priorities)
/DeepenData/Repos/Flux_v0/src/utils_Escritoriosv05_Simv05.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Priority'] = df['IdSerie'].map(priorities)
/DeepenData/Repos/Flux_v0/src/utils_Escritoriosv05_Simv05.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = 

## IN PROGRESS : OPTUNA 

# Objetivos optuna (broken)

In [ ]:

import optuna
import numpy as np
dataset = DatasetTTP.desde_csv_atenciones("data/fonasa_monjitas.csv.gz")
un_dia = dataset.un_dia("2023-05-15").sort_values(by='FH_Emi', inplace=False)
skills   = obtener_skills(un_dia)
series   = sorted(list({val for sublist in skills.values() for val in sublist}))
modos    = ['Rebalse','Alternancia', 'Rebalse']
atributos_series = atributos_x_serie(ids_series=series, 
                                    sla_porcen_user=None, 
                                    sla_corte_user=None, 
                                    pasos_user=None, 
                                    prioridades_user=None)

niveles_servicio_x_serie = {atr_dict['serie']:
                           (atr_dict['sla_porcen']/100, atr_dict['sla_corte']/60) 
                           for atr_dict in atributos_series}

prioridades =       {atr_dict['serie']:
                    atr_dict['prioridad']
                    for atr_dict in atributos_series}

def objective(trial, 
    optimizar: str, 
    un_dia : pd.DataFrame,  # IdOficina  IdSerie  IdEsc, FH_Emi, FH_Llama  -- Deberia llamarse 'un_tramo'
    subsets, # [(5,), (10,), (11,), (12,), (14,), (17,), (5, 10), (5, 11), (5, 12), (5, 14), (5, 17), (10, 11),  <...> 14, 17), (5, 10, 12, 14, 17), (5, 11, 12, 14, 17), (10, 11, 12, 14, 17), (5, 10, 11, 12, 14, 17)]
    niveles_servicio_x_serie,  # {5: (0.34, 35), 10: (0.34, 35), 11: (0.7, 45), 12: (0.34, 35), 14: (0.34, 35), 17: (0.6, 30)}
    prioridades:dict,
    modos_atenciones : list = ["Alternancia", "FIFO", "Rebalse"],
    minimo_escritorios: int = 2,
    maximo_escritorios: int = 5,
    ):    
    try:

        bool_vector              = [trial.suggest_categorical(f'escritorio_{i}', [True, False]) for i in range(maximo_escritorios)]
        #Restricción de minimo de escritorios
        assert sum(bool_vector) >= minimo_escritorios, f"No cumple con minimo_escritorios: {minimo_escritorios}."
        
        str_dict                 = {i: trial.suggest_categorical(f'{i}',         modos_atenciones) for i in range(maximo_escritorios)} 
        subset_idx               = {i: trial.suggest_int(f'ids_{i}', 0, len(subsets) - 1) for i in range(maximo_escritorios)}   
        #prioridades              =  prioridad_x_serie(niveles_servicio_x_serie, 2, 1) 
        planificacion            =  {} # Arma una planificacion con espacios parametricos. 
        inicio                   =  str(un_dia.FH_Emi.min().time())#'08:33:00'
        termino                  =  str(un_dia.FH_Emi.max().time())#'14:33:00'
        #porcentaje_actividad     =  np.random.randint(75, 90)/100
        for key in str_dict.keys():
            if bool_vector[key]:
                inner_dict = {
                    'inicio': inicio,
                    'termino': termino,
                    'propiedades': {
                        'skills':list(subsets[subset_idx[key]]), # Set -> Lista, para el subset 'subset_idx', para el escritorio 'key'
                        'configuracion_atencion': str_dict[key], # 
                    }
                }
                planificacion[str(key)] = [inner_dict] # NOTE: Es una lista why -- Config por trial por tramo del escritorio 

        trial.set_user_attr('planificacion', planificacion) # This' actually cool 
        registros_atenciones, l_fila    =  optuna_simular_v02(planificacion, niveles_servicio_x_serie, un_dia, prioridades, tipo_inactividad = "historica") 
        registros_atenciones['IdSerie'] = registros_atenciones['IdSerie'].astype(int) 
        registros_x_serie               = [registros_atenciones[registros_atenciones.IdSerie==s] for s in series]
        
        
        pocentajes_SLA        = [int(100*v[0])for k,v in niveles_servicio_x_serie.items()]
        mins_de_corte_SLA     = [int(v[1])for k,v in niveles_servicio_x_serie.items()]        
        df_pairs              = [(sla_x_serie(r_x_s, '1H', corte = corte, factor_conversion_T_esp=1), s) 
                                    for r_x_s, s, corte in zip(registros_x_serie, series, mins_de_corte_SLA)]
        porcentajes_reales    = {f"serie: {serie}": np.mean(esperas.espera) for ((demandas, esperas), serie) in df_pairs} 
        dif_cuadratica        = {k:(v-p)**2 for ((k,v),p) in zip(porcentajes_reales.items(),pocentajes_SLA)}
        #Objetivos:    
        #La mayor prioridad es el entero más chico    
        maximizar_SLAs        = tuple(np.array(tuple(prioridades.values()))*np.array(tuple(dif_cuadratica.values())))#Ponderado por prioridad
        minimizar_escritorios = (sum(bool_vector),)
        minimizar_skills      = (extract_skills_length(planificacion),)
        
        if optimizar == "SLA":
            
            print(f"maximizar_SLAs {maximizar_SLAs}")
            return  maximizar_SLAs
        
        elif optimizar == "SLA + escritorios":
            
            print(f"maximizar_SLAs y minimizar_escritorios {maximizar_SLAs, minimizar_escritorios}")
            return  maximizar_SLAs + minimizar_escritorios
        
        elif optimizar == "SLA + skills":
            
            print(f"maximizar_SLAs y minimizar_skills {maximizar_SLAs, minimizar_skills}")
            return  maximizar_SLAs + minimizar_skills
        
        elif optimizar == "SLA + escritorios + skills":
            
            print(f"SLA + escritorios + skills {maximizar_SLAs, minimizar_escritorios, minimizar_skills}")
            return  maximizar_SLAs + minimizar_escritorios + minimizar_skills           
        
    except Exception as e:
        print(f"An exception occurred: {e}")
        raise optuna.TrialPruned()
    
#Si hay porcentaje_actividad no hay pausas  
#n es el numero de intervalos (equidistantes) de tiempo   
intervals  = get_time_intervals(un_dia, n = 4, porcentaje_actividad = .9) # Una funcion que recibe un dia, un intervalo, y un porcentaje de actividad para todos los intervalos
partitions = partition_dataframe_by_time_intervals(un_dia, intervals) # TODO: implementar como un static del simulador? 
optimizar  = "SLA + escritorios + skills" #"SLA" | "SLA + escritorios" | "SLA + skills" | "SLA + escritorios + skills"
n_objs = int(
            len(series)
            if optimizar == "SLA"
            else len(series) + 1
            if optimizar in {"SLA + escritorios", "SLA + skills"}
            else len(series) + 2
            if optimizar == "SLA + escritorios + skills"
            else None
        )
n_trials   = 1
#%%
storage = optuna.storages.get_storage("sqlite:///alejandro_objs_v3.db")
subsets = non_empty_subsets(sorted(list({val for sublist in skills.values() for val in sublist})))

for idx, part in enumerate(partitions):
    study_name = f"tramo_{idx}"
    study = optuna.multi_objective.create_study(directions= n_objs*['minimize'],
                                                study_name=study_name,
                                                storage=storage, load_if_exists=True)
    # TODO: sacar fuera
    # Optimize with a timeout (in seconds)
    study.optimize(lambda trial: objective(trial,
                                           optimizar                = optimizar,
                                           un_dia                   = part,
                                           subsets                  = subsets,
                                           niveles_servicio_x_serie = niveles_servicio_x_serie,
                                           prioridades              = prioridades,
                                           minimo_escritorios       = 2,
                                           maximo_escritorios       = 5
                                           ),
                   n_trials  = n_trials, #int(1e4),  # Make sure this is an integer
                   timeout   = 2*3600,   #  hours
                   )  # 


In [ ]:
study.get_trials()[0].user_attrs.get('planificacion')

## Extracción de la planificación óptima

In [ ]:
recomendaciones_db   = optuna.storages.get_storage("sqlite:///alejandro_objs_v3.db") # Objetivos de 6-salidas
resumenes            = optuna.study.get_all_study_summaries(recomendaciones_db)
nombres              = [s.study_name for s in resumenes if "tramo_" in s.study_name]

scores_studios = {}
for un_nombre in nombres:
    un_estudio            = optuna.multi_objective.load_study(study_name=un_nombre, storage=recomendaciones_db)
    trials_de_un_estudio  = un_estudio.get_trials(deepcopy=False) #or pareto trials??
    scores_studios        = scores_studios | {f"{un_nombre}":
        { trial.number: np.mean([x for x in trial.values if x is not None]) 
                for
                    trial in trials_de_un_estudio if trial.state == optuna.trial.TrialState.COMPLETE}
                    } 
trials_optimos          = extract_min_value_keys(scores_studios) # Para cada tramo, extrae el maximo, 
planificaciones_optimas = {}   
for k,v in trials_optimos.items():
    un_estudio               = optuna.multi_objective.load_study(study_name=k, storage=recomendaciones_db)
    trials_de_un_estudio     = un_estudio.get_trials(deepcopy=False)
    planificaciones_optimas  = planificaciones_optimas | {f"{k}":
        trial.user_attrs.get('planificacion')#calcular_optimo(trial.values)
                for
                    trial in trials_de_un_estudio if trial.number == v[0]
                    }   

planificacion_optima                =  plan_unico([plan for tramo,plan in planificaciones_optimas.items()])


### simulación con planificación óptima

In [ ]:

def ghost_client(un_dia, slack_seconds=7200):
    
    un_dia.sort_values(by='FH_Emi', inplace=True)
    un_dia.reset_index(drop=True, inplace=True)
    last_row = un_dia.iloc[-1].copy()
    # Modify datetime64[s] columns by adding 2 hours (7200 seconds)
    datetime_columns = ['FH_Emi', 'FH_Llama', 'FH_AteIni', 'FH_AteFin']
    for col in datetime_columns:
        last_row[col] += pd.Timedelta(seconds=slack_seconds)  # 2 hours = 7200 seconds

    return pd.concat([un_dia, pd.DataFrame(last_row).T], axis=0, ignore_index=True)



In [ ]:
registros_atenciones_optima, l_fila_optima =  optuna_simular_v02(planificacion_optima, niveles_servicio_x_serie, un_dia_con_ghost_client, prioridades, tipo_inactividad = "historica") 

In [ ]:
registros_atenciones_optima, l_fila_optima

In [ ]:
registros_atenciones_optima["IdEsc"].info()